# Legiscan Script (unusable without your own API key)
##### Using Legiscan.com API
## Create list of bills about a topic of your choice
#### -This script customized for bills about MeToo
##### -----creates field for when bill was last acted on in relation to the NYT story about Weinstein in Oct. 2017
##### -----current implentation adds _NEW to bills added to the list during the search (which need to be replaced manually) 
#### -Use multiple queries without getting overlap
#### -Gives option to review the bills for match as they are written to the spreadsheet
#### -Keeps bill progress up to date if fount in a search

## By Aaron Hegarty, USAT intern summer 2018
### (402) 317-2702
### aphegarty@huskers.unl.edu

## Step by step instructions

### To begin
#### >From your excel file, save as a new file - a csv. Do not save over your excel file.
#### >Make sure you have master_list.csv in the same folder as this .ipynb file.
#### >master_list.csv must be a CSV, not an excel file, such as .xlsx
#### Run all above from 'GO BLOCK' below
____________________________________________________________
### To run a search
#### >Run all below from 'GO BLOCK' below
#### >Follow the prompts as you go.
____________________________________________________________
### When done
#### >Go to master_list.csv and copy and paste all rows who have a value of _NEW or TODO
#### > When copied over, replace all _NEW with nothing if you choose to run updates again
____________________________________________________________
### If you run into an 'encoding' error
#### >God help you if you do this is annoying
#### >this is why a copy is automatically made
#### >Note the master_list file likely has to be replaced with the most recent file if the error occured after adding the bills, but the master_list likely has at least the information you worked on. Keep this and clean it up
#### >make sure csv file in encoded in UTF-8. Open master_list.csv in a program and convert to UTF-8. On a PC, this is easy to do in Notepad++.
_____________________________________________________________
### To update all bills
#### >See update_all script

In [ ]:
#imports 
import urllib.parse #make query in url form
import requests
import csv
import pandas as pd #reads in csv, makes lists of existing bills, and helps identify bills that need to be updated
import datetime
from shutil import copyfile #copy file in case of disastor

### Functions used

In [ ]:
#runs a request using the API to get for information about the bill. Returns a list to be used to write the bill
def getBillDetes(bill_id):
    bill_url = legisearch_1 + '&op=getBill&id=' + bill_id
    getBill = requests.get(bill_url) #Legiscan!
    getBill_JSON = getBill.json() #do json stuff i don't really understand
    state = getBill_JSON['bill']['state']
    name = getBill_JSON['bill']['bill_number']
    status_code = str(getBill_JSON['bill']['status'])
    try:
        status = status_list[status_code]
    except:
        status = "Unknown"
    last_action = convertDate(getBill_JSON['bill']['status_date'])
    title = getBill_JSON['bill']['title']
    link = getBill_JSON['bill']['url']
    print("-----------------------------------------------")
    print("Bill Title:", title, "(" + str(bill_id) + ")")
    print("Relevance Score: " + str(relevance))
    print("More: " + str(link))
    return([state, name, status, last_action, title, link])

In [ ]:
#used to write bills that did not exist in our list and are reviewed OR are being updated
    #uses meToo function to determine the period used
def writeBill(state, name, status, last_action, title, link, match, notes):
    bills_written.append(bill_id)
    writer.writerow({
    'bill_id': bill_id, 'match' : match, 'period':meToo(last_action),
    'relevance': relevance, 'state': state,
    'name': name, 'status': status,
    'last_action': last_action, 'title': title,
    'link': link, 'change_hash':change_hash_id,
    'bill_update': today, 'query_used_latest':query,
    'notes':notes}
    )

In [ ]:
#Used to write bills that were already in our list and are not being updated. 
    #Does not use #meToo function to find the period we will clasify it as since we already did that
    #instead takes period as input to the function
def writeBillComplete(state, name, status, last_action, title, link, match, notes,period):
    bills_written.append(bill_id)
    writer.writerow({
    'bill_id': bill_id, 'match' : match, 'period':period,
    'relevance': relevance, 'state': state,
    'name': name, 'status': status,
    'last_action': last_action, 'title': title,
    'link': link, 'change_hash':change_hash_id,
    'bill_update': today, 'query_used_latest':query,
    'notes':notes}
    )

In [ ]:
#determines period we classified it as
    #After Oct. 4, 2017 - "after" - after NYT story
    #also indicates if part of the year prior. This could be used to show the increase in bills IF you've reviewed results from quieries in those periods as well. You can change the year part of the query to gaurantee this
def meToo(string_date):
    string_date = convertDate(string_date)
    if string_date > "2017-10-04":
        return("after")
    elif string_date < "2017-10-05" and string_date > "2016-10-04":
        return("1_year_prior")
    elif string_date < "2016-10-05" and string_date > "2015-10-04":
        return("2_year_prior")
    else:
        return("well_before")

In [ ]:
def getRuling(state, name, status, last_action, title, link): 
    print("Getting ruling...")
    while True:
        user_input = input("Is this about #MeToo? Type Y, N, M, PN, PY, or Q to quit.")
        if user_input == "Y":
            print(">>>>>>>>>>>Yay! A match!")
            break
        elif user_input == "N":
            print(">>>>>>>>>>>Ope! Sorry! Not about #MeToo...")
            break
        elif user_input == "M":
            print(">>>>>>>>>>>Marked as Maybe")
            break
        elif user_input == "PN":
            print(">>>>>>>>>>>Marked as probably not.")
            break
        elif user_input == "PY":
            print(">>>>>>>>>>>Marked as probably a match")
            break
        elif user_input == "CY":
            print(">>>>>>>>>>>Marked as a companion bill yes")
            break
        elif user_input == "Q":
            outfile.close()
            print("-----------------------------------------------")
            print("QUITING.... see master_list.csv to see your work.")
            return(False)
        else:
            print("ENTER Y, N, M, PN, PY, or Q to quit!!!")
    notes_input = input("Notes: ")
    writeBill(state, name, status, last_action, title, link, user_input+"_NEW", notes_input)
    #writeBill(state, name, status, last_action, title, link, user_input, notes_input) 
    return(True)

In [ ]:
#convertDate used in meToo function
def convertDate(s_date):
    date_patterns = ["%m/%d/%Y", "%Y-%m-%d"]
    for pattern in date_patterns:
        try:
            day = datetime.datetime.strptime(s_date, pattern)
            return day.strftime('%Y-%m-%d')
        except:
            pass

### API stuff. 

In [ ]:
apikey = "REMOVED" 
legisearch_1 = "https://api.legiscan.com/?key=" + apikey
#define status 
status_list = {'1' : 'Introduced',
'2' : 'Engrossed',
'3' : 'Enrolled',
'4' : 'Passed',
'5' : 'Vetoed',
'6' : 'Failed',
'7' : 'Override',
'8' : 'Chaptered',
'9' : 'Refer',
'10' : 'Report Pass',
'11' : 'Report DNP',
'12' : 'Draft'}

# -----GO BLOCK-----
## THINGS THAT NEED TO BE RERUN TO RUN A NEW QUERY ARE BELOW
### If first search since opening: Click on this cell > Cell > Run all above
### To run a search: Click on this cell > Cell > Run all below

In [ ]:
now = datetime.datetime.now()
today = now.strftime("%Y-%m-%d")
time_stamp = now.strftime("%Y-%m-%d-at-%H%M")
existing_file = 'master_list.csv'

In [ ]:
query_input = input("What's the query you want to use? Type status:passed after to search for only passed bills.")
query = query_input
#query = "\"sexual abuse\" status:passed" #enter query
year_input = input("How far back do you want to look? 1=all, 2=current, 3=recent, 4=prior, >1900=exact [Default: 2]. TO GET ALL SINCE Oct 2017 you have to use '3' Bills before Oct. 4. 2016 will be skipped.")
year = year_input #Year where 1=all, 2=current, 3=recent, 4=prior, >1900=exact [Default: 2]
query_url_encode = urllib.parse.quote_plus(query) #url encode
state = input("What state? Type the 2 letter abreviation or 'all'")
search_url = legisearch_1 + "&op=searchRaw&state=" + state + "&query=" + query_url_encode + "&year=" + year
print(search_url) #this is what will run

In [ ]:
#get list of bills we already have
existing = pd.read_csv(existing_file, encoding='utf-8', dtype={'bill_id':'str', 'change_hash':'str'}, index_col = 'change_hash')
change_hash_list = existing.index.tolist()
bill_id_list = existing['bill_id'].tolist()
bill_checker = dict(zip(bill_id_list, change_hash_list))

In [ ]:
r_search = requests.get(search_url) #Legiscan!
matching_json = r_search.json() #do json stuff i don't really understand
matches = matching_json['searchresult']['results'] #list of results
print(matching_json['searchresult']['summary']) #print summary
bill_count = matching_json['searchresult']['summary']['count']

In [ ]:
#indicatite whether you want to make decisions while the code is running.
while True:
    print("\n\nIndicatite whether you want to make decisions while the code is running....\n")
    print("If N, all bills from this search that are not already in the list will be added at the top with TODO or OLD in the match column.")
    print("If Y, you will be given the option to quit while reviewing. It skips bills older than oct. 4, 2015.\n")
    review_question = "Do you want to review all " + str(bill_count) + " bills? Type Y or N."
    user_input = input(review_question)
    if user_input == "Y":
        print(">>>>>>>>>>>REVIEWING ALL AS YOU GO. HIT Q TO QUIT when making a decision.")
        review = True
        break
    elif user_input == "N":
        print(">>>>>>>>>>>NOT REVIEWING. All bills from this search that are not already in the list will be added near the top with TODO in the match column.")
        review = False
        break
    else:
        print(review_question)
        print("ENTER Y for YES or N for NO.")

In [ ]:
with open('master_list.csv', 'w', newline='', encoding='utf-8') as outfile: #w for overwriting, NOT a for appending. 
    #All bills that are not included in the search will be added using pandas at the end of this script 
    print("Opening file....")
    writer = csv.DictWriter(outfile, fieldnames=['change_hash', 'bill_id', 'match', 'period', 'relevance', 'state', 'name', 'status', 'last_action', 'title', 'link', 'bill_update', 'query_used_latest', 'notes'])
    writer.writeheader() 
    print("writing header")
    bill_count = matching_json['searchresult']['summary']['count']
    #Create query the will be logged in the file. Provides information about the year and state used in the search and the number of bills it returned.
    query = query + "_YR:" + str(year) + "_count:" + str(bill_count) +"ST:" +str(state)
    #The script will only deal with the first 2,000 results in a search. If longer, consider changing the query
    if bill_count > 2000:
        bill_count = 2000
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("Does not support results after 2000, change the page in the legiscan API search query using '&page=' if necessary")
    else: 
        bill_count = bill_count
    #bills_written keeps track of what we've written so there's no overlap in resulting file. At the end, write every bill that is not in this list.
    bills_written = [] 
    ruling = True #this can be changed to false in the getRuling function in order to exit loop by entering 'Q'
    for i in range(0,bill_count): #for each bill in search result
        print("-----------------------------------------------")
        print("Result: ", i)
        relevance = matches[i]['relevance'] #How well that bill might match to your query
        bill_id = str(matches[i]['bill_id']) #Provided by Legiscan
        change_hash_id = str(matches[i]['change_hash']) #Used to detect changes to bills, update
        #If bill has not been reviewed and therefore is not in master_list.csv
        if not (bill_id in bill_id_list): #if has not been found in a previous search
            detes = getBillDetes(bill_id) #get details like bill name, title, status, etc
            last_action = convertDate(detes[3]) #convert to YYYY-MM-DD
            print("State: ", detes[0])
            print("Status: ", detes[2])
            if last_action < "2016-10-04": #if more than a year before the NYT story 
                print("Skipping because OLD (before 2016-10-04) and writing OLD to indicate not reviewed and it matches...")
                writeBill(detes[0], detes[1], detes[2], last_action, detes[4], detes[5], "OLD", "Not reviewed initially")
            elif review == False:
                print("Skipping, but writing TODO to indicate not reviewed and it matches and is not too old (before 2016-10-04)...")
                writeBill(detes[0], detes[1], detes[2], last_action, detes[4], detes[5], "TODO", "Not reviewed initially")
            else:
                ruling = getRuling(detes[0], detes[1], detes[2], last_action, detes[4], detes[5])
            if ruling is False: break #if 'Q' was entered, break and write all bills that are not already written
        elif bill_checker[bill_id] != change_hash_id: #if needs updating
            print("Update needed on", bill_id, "..................UPDATING.................")
            detes = getBillDetes(bill_id)
            writeBill(detes[0], detes[1], detes[2], detes[3], detes[4], detes[5], existing.at[bill_checker[bill_id], 'match'], (existing.at[bill_checker[bill_id], 'notes']))
            print("Updated", bill_id + ".")
        else: #already existed as is
            print("You already handled", bill_id, "and there is no need to update. If you need to change decision, edit in a spreadsheet.")
            #rewrite row
            writeBillComplete(existing.at[change_hash_id, 'state'], existing.at[change_hash_id, 'name'], 
                      existing.at[change_hash_id, 'status'], existing.at[change_hash_id, 'last_action'], 
                      existing.at[change_hash_id, 'title'], existing.at[change_hash_id, 'link'], 
                      existing.at[change_hash_id, 'match'], existing.at[change_hash_id, 'notes'], existing.at[change_hash_id, 'period'])
outfile.close()
#Write every bill that hasn't been written yet 
existing_left = existing.loc[~existing['bill_id'].isin(bills_written)]
with open('master_list.csv', 'a', encoding='utf-8') as outfile2:
    existing_left.to_csv(outfile2, encoding='utf-8', header=False)
outfile2.close()
copyfile('master_list.csv', 'master_list' + time_stamp + '.csv') #copies original file as original_...